#Algorithme de Berstein-Vazirani

Importations de librairies

In [25]:
from qiskit import QuantumCircuit
from FonctionsCommune import *
import numpy as np

Fonctions

Fonction de création des oracles

In [26]:
'''Nous avons décidé de ne pas inclure le nombres de qubits en argument car nous
 avons changé la fonction decimal_in_bit_string qui n'en a donc plus besoin.'''

def OracleBersteinVazirani(decimal):

    bits_string = decimal_in_bit_string(decimal)
    string_length = len(bits_string)

    qc = QuantumCircuit(string_length + 1, string_length)

    string_length = len(bits_string)

    for i in range(string_length):
        if bits_string[i] == '1':
            qc.cx(i, string_length)

    return qc

Fonction pour le circuit de Berstein-Vazirani

In [27]:
def BersteinVazirani(decimal):

    bits_string = decimal_in_bit_string(decimal)
    string_length = len(bits_string)

    qc = QuantumCircuit(string_length + 1, string_length)

    for i in range(string_length):
        qc.h(i)

    qc.append(OracleBersteinVazirani(decimal), range(0, string_length+1))

    for i in range(string_length):
        qc.h(i)

    return qc

Éxecution du circuit

In [28]:
decimals = [np.random.randint(0,50) for i in range(10)]
bit_string_result = []

for decimal in decimals:

    Bernstein_vazirani_circuit = BersteinVazirani(decimal)
    Bernstein_vazirani_circuit.measure(range(Bernstein_vazirani_circuit.num_qubits-1),
                                       range(Bernstein_vazirani_circuit.num_clbits))

    current_bit_string = []

    for i in range (Bernstein_vazirani_circuit.num_clbits):
        current_bit_string.append(Bernstein_vazirani_circuit.clbits[i])

    bit_string_result.append(current_bit_string)

for i in range(len(bit_string_result)):
    print(decimals[i])
    print(bit_string_result[i])

TypeError: decimal_in_bit_string() missing 1 required positional argument: 'nb_qbits'